In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

2025-02-02 00:46:34.762803: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738457194.775097    1389 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738457194.778491    1389 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-02 00:46:34.791117: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found GPU at: /device:GPU:0


I0000 00:00:1738457196.289340    1389 gpu_device.cc:2022] Created device /device:GPU:0 with 5592 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


In [ ]:
import tensorflow as tf
import timeit


device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')



def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((200, 300, 300, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((200, 300, 300, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 200x300x300x6 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

I0000 00:00:1738457197.000495    1389 gpu_device.cc:2022] Created device /device:GPU:0 with 5592 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6
I0000 00:00:1738457197.001253    1389 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5592 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6
2025-02-02 00:46:37.242111: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2212761600 exceeds 10% of free system memory.
2025-02-02 00:46:37.699756: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2212761600 exceeds 10% of free system memory.
I0000 00:00:1738457198.735334    1389 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-02-02 00:46:39.704327: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:306] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.28GiB with freed_

Time (s) to convolve 32x7x7x3 filter over random 200x200x200x6 images (batch x height x width x channel). Sum of ten runs.
CPU (s):


2025-02-02 00:46:40.941307: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2212761600 exceeds 10% of free system memory.
2025-02-02 00:46:41.383747: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2212761600 exceeds 10% of free system memory.
2025-02-02 00:46:41.916997: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2212761600 exceeds 10% of free system memory.


9.24929323700053
GPU (s):
0.7872460540002066
GPU speedup over CPU: 11x


In [ ]:
# This is a very intensive test that will sometimes fail to run. While this now proves the speed of your GPU, it also proves that 8GB of VRAM is insufficient.
# Perhaps it will work, but from the beginning it was always a question of the dataset.